In [1]:
from datasets import load_dataset

In [2]:
ds = load_dataset("./source/load_SCOTUS.py",trust_remote_code=True)
dstrain = ds["train"]
dsval = ds["validation"]


In [ ]:
print(len(dstrain)+ len(dsval))

In [ ]:
link = ds[0]["justia_link"]
print([ ex["justia_link"] for ex in ds])
print(link)

In [3]:
url = "https://supreme.justia.com/cases/federal/us/539/444/"
ds = ds["train"]

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from selenium.webdriver.chrome.service import Service
from source.utils import get_file
from tqdm.notebook import tqdm
import json
os.makedirs('/home/labicquette/M2/projet/data/audios', exist_ok=True)

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", {
    'download.default_directory': '/home/labicquette/M2/projet/data/audios',
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True,
    "download.default_directory": '/home/labicquette/M2/projet/data/audios',
    "savefile.default_directory": '/home/labicquette/M2/projet/data/audios',

})
chrome_options.add_argument("--headless=new")


#print(driver.capabilities)

articles = []
#for opi in tqdm(range(138,len(ds))):

def scrap(opi):

    if os.path.exists("./data/audios/train/"+get_file(opi, "SCOTUS")+".mp3"):
        return 
    driver = webdriver.Chrome(options=chrome_options)
    #opi = ds[opi]
    url = opi["justia_link"]
    #print(opi["docket_number"])
    #print(opi["year_argued"])
    #print(url)
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://www.oyez.org/cases/"+opi["year_argued"]+"/"+opi["docket_number"])


    try :
        link = WebDriverWait(driver, 4).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(@iframe-url, 'oral_argument_audio')]"))
        )
    except:
        driver.close()
        driver.quit()
        return
       
    link = driver.find_element(By.XPATH, "//a[contains(@iframe-url, 'oral_argument_audio')]")
    link.click()

    outHTML = link.get_attribute("outerHTML")

    id_audio = outHTML.split("oral_argument_audio - ")[1].split('"')[0]

    response = requests.get("https://api.oyez.org/case_media/oral_argument_audio/"+id_audio+"/download")
    
    file_Path = "./data/audios/train/"+get_file(opi, "SCOTUS")+".mp3"

    if response.status_code == 200:
        #print(response)
        with open(file_Path, 'wb') as file:
            file.write(response.content)
        #print('File downloaded successfully')
    else:
        print('Failed to download file')


    #driver.implicitly_wait(5)
    #TODO PROBLEM OF TAG MISMATCH
    #contents =driver.find_element(By.CLASS_NAME, "transcript")
    


    #elem = driver.find_elements(By.CLASS_NAME, "transcript-section ng-scope")
    #print(elem)
    #print(type(elem.get_attribute("innerHTML")))

    #print(len(article))
    #print(article[0].get_attribute("transcript"))
    #transcript = article[0].get_attribute("outerHTML")
#/html/body/div[2]/div/div/article
    #file_Path = "./data/transtest/test.json"
    #file_Path = "./data/transcripts/train/"+get_file(opi, "SCOTUS")+".json"
    #with open(file_Path, "w") as outfile: 
    #    json.dump({"conversations":opi_dict}, outfile)
    

    #driver.get("https://api.oyez.org/case_media/oral_argument_audio/"+id_audio+"/download")
    driver.close()
    driver.quit()

ds.map(lambda example : scrap(example), num_proc=4)

Map (num_proc=4):   0%|          | 0/2670 [00:00<?, ? examples/s]

Failed to download file


Dataset({
    features: ['justia_link', 'docket_number', 'year_argued', 'court_name', 'majority_opinion_author', 'raw_source', 'opinion_texts_source', 'raw_target', 'case_name', 'description', 'first_party_label', 'second_party_label', 'decisions', 'written_opinion', 'examples', 'labels'],
    num_rows: 2670
})

In [ ]:
print(outHTML.split("oral_argument_audio - ")[1].split('"')[0])

In [ ]:
print(article[0].get_attribute("innerHTML"))

In [ ]:
print(article[0].get_attribute("outerHTML"))

In [ ]:
#for a in articles:
print(articles[0][0].get_attribute("outerHTML"))

In [ ]:
"https://api.oyez.org/case_media/oral_argument_audio/23600/download"